<a href="https://colab.research.google.com/github/starryesh22/Google_Colab/blob/main/0608_Shap_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import argparse
import os

import numpy as np
import pandas as pd
import torch
import shap
from tqdm import tqdm
import matplotlib.pyplot as plt

from resnet import resnet34
from utils import prepare_input


In [ ]:
'''

argparse: 명령줄 인자 파싱을 위한 모듈입니다. 명령줄에서 인자를 읽고 파싱하는 기능을 제공합니다.
os: 운영 체제와 상호 작용하기 위한 모듈입니다. 디렉토리 생성, 파일 경로 조작 등의 기능을 제공합니다.
numpy: 수치 연산을 위한 라이브러리입니다. 다차원 배열과 행렬 연산에 유용한 기능을 제공합니다.
pandas: 데이터 조작과 분석을 위한 라이브러리입니다. 데이터를 효과적으로 다루고 조작할 수 있는 기능을 제공합니다.
torch: 파이토치 머신 러닝 프레임워크입니다. 텐서 계산, 자동 미분, 신경망 모델 등을 구현할 수 있습니다.
shap: SHAP (SHapley Additive exPlanations) 값을 계산하는 라이브러리입니다. 머신 러닝 모델의 특성 중요도를 설명하는 데 사용됩니다.
tqdm: 반복문의 진행 상태를 시각적으로 표시하는 라이브러리입니다. 진행률 바, 소요 시간 등을 제공합니다.
matplotlib.pyplot: 시각화를 위한 라이브러리입니다. 그래프와 플롯을 생성하고 시각적인 요소를 추가할 수 있습니다.
resnet: resnet34 모델을 정의한 모듈입니다. ResNet은 심층 신경망 아키텍처로 이미지 분류 등에 사용됩니다.
utils: 여러 가지 유틸리티 함수가 포함된 모듈입니다. 예를 들어, 입력 데이터를 준비하는 기능을 제공합니다.


'''

In [ ]:
# parse_args 함수

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-dir', type=str, default='data/CPSC', help='Data directory')
    parser.add_argument('--leads', type=str, default='all')
    parser.add_argument('--seed', type=int, default=42, help='Seed to split data')
    parser.add_argument('--use-gpu', default=False, action='store_true', help='Use GPU')
    return parser.parse_args()


'''

제공한 코드는 argparse를 사용하여 명령줄 인자를 파싱하는 parse_args 함수입니다. 이 함수는 다음과 같은 인자를 받습니다:

--data-dir: 데이터 디렉토리의 경로를 지정합니다. 기본값은 'data/CPSC'입니다.
--leads: 사용할 리드(심전도 신호)를 지정합니다. 기본값은 'all'입니다.
--seed: 데이터를 분할할 때 사용할 시드 값을 지정합니다. 기본값은 42입니다.
--use-gpu: GPU를 사용할지 여부를 지정합니다. 기본값은 False입니다.
이 함수는 argparse.ArgumentParser 객체를 생성하고, 각 인자에 대한 정보를 추가합니다.
 --data-dir, --leads, --seed, --use-gpu와 같은 인자들은 명령줄에서 지정할 수 있는 인자들입니다.

parser.parse_args()를 호출하여 명령줄 인자를 파싱하고, 해당 인자들의 값을 반환합니다.

이 함수를 사용하면 다음과 같이 명령줄에서 인자를 지정하여 프로그램을 실행할 수 있습니다:

'''



In [ ]:
# plot_shap 함수

def plot_shap(ecg_data, sv_data, top_leads, patient_id, label):
    # patient-level interpretation along with raw ECG data
    leads = np.array(['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    nleads = len(top_leads)
    if nleads == 0:
        return
    nsteps = 5000 # ecg_data.shape[1], visualize last 10 s since many patients' ECG are <=10 s
    x = range(nsteps)
    ecg_data = ecg_data[:, -nsteps:]
    sv_data = sv_data[:, -nsteps:]
    threshold = 0.001 # set threshold to highlight features with high shap values
    fig, axs = plt.subplots(nleads, figsize=(9, nleads))
    fig.suptitle(label)
    for i, lead in enumerate(top_leads):
        sv_upper = np.ma.masked_where(sv_data[lead] >= threshold, ecg_data[lead])
        sv_lower = np.ma.masked_where(sv_data[lead] < threshold, ecg_data[lead])
        if nleads == 1:
            axe = axs
        else:
            axe = axs[i]
        axe.plot(x, sv_upper, x, sv_lower)
        axe.set_xticks([])
        axe.set_yticks([])
        axe.set_ylabel(leads[lead])
    plt.savefig(f'shap/shap1-{patient_id}.png')
    plt.close(fig)


In [ ]:
'''plot_shap 함수로, SHAP 값을 시각화하고 환자의 원시 ECG 데이터와 함께 표시하는 역할을 합니다. 이 함수는 다음 매개변수를 사용합니다:

ecg_data: 원시 ECG 데이터입니다.
sv_data: SHAP 값을 포함하는 ECG 데이터입니다.
top_leads: 상위 리드(심전도 신호)의 인덱스 리스트입니다.
patient_id: 환자의 ID입니다.
label: 시각화의 제목(label)입니다.
함수의 동작은 다음과 같습니다:

상위 리드(top_leads)를 기반으로 시각화할 리드 수(nleads)를 결정합니다. 만약 nleads가 0이면 함수를 종료합니다.
시각화할 데이터의 시간 스텝 수(nsteps)를 설정합니다. 주석에는 ecg_data.shape[1]로 주석 처리되어 있지만, 실제로는 마지막 10초만 시각화하기 위해 5000으로 설정되어 있습니다. (많은 환자들의 ECG 데이터가 10초 이하일 수 있기 때문입니다)
시각화할 데이터를 마지막 nsteps만 남기도록 잘라냅니다.
SHAP 값이 일정 임계값(threshold)보다 큰지 여부에 따라 ECG 데이터를 상·하로 분할하여 시각적으로 강조합니다.
상위 리드(top_leads)의 개수에 따라 서브플롯을 생성하고, 각 리드에 대한 데이터를 그립니다.
각 서브플롯의 축 눈금을 설정합니다.
시각화를 파일로 저장합니다.
이 함수를 사용하면 SHAP 값을 시각화하고, 강조된 특성과 함께 환자의 ECG 데이터를 확인할 수 있습니다. 그림은 shap 폴더에 shap1-{patient_id}.png로 저장됩니다.


'''


In [ ]:
# summary_plot 함수

def summary_plot(svs, y_scores):
    leads = np.array(['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    svs2 = []
    n = y_scores.shape[0]
    for i in tqdm(range(n)):
        label = np.argmax(y_scores[i])
        sv_data = svs[label, i]
        svs2.append(np.sum(sv_data, axis=1))
    svs2 = np.vstack(svs2)
    svs_data = np.mean(svs2, axis=0)
    plt.plot(leads, svs_data)
    plt.savefig('./shap/summary.png')
    plt.clf()




In [ ]:
''' 제공한 코드는 summary_plot 함수로, SHAP 값을 요약하고 시각화하는 역할을 합니다. 이 함수는 다음과 같은 매개변수를 사용합니다:

svs: SHAP 값들을 포함하는 배열입니다.
y_scores: 예측된 클래스 점수들을 포함하는 배열입니다.
함수의 동작은 다음과 같습니다:

리드(심전도 신호)의 배열(leads)을 생성합니다.
SHAP 값을 요약하기 위한 배열(svs2)을 초기화합니다.
예측된 클래스 점수들(y_scores)의 개수(n)만큼 반복하면서, 각 샘플에 대한 SHAP 값을 추출하고 요약합니다.
요약된 SHAP 값들(svs2)을 수직으로 쌓아올립니다.
SHAP 값을 평균하여 요약된 SHAP 데이터를 생성합니다.
리드와 해당 리드에 대한 SHAP 값들을 시각화합니다.
시각화를 './shap/summary.png' 파일로 저장합니다.
그림을 초기화합니다.
이 함수를 사용하면 SHAP 값의 요약 정보를 시각화하여 전체적인 특성 중요도를 파악할 수 있습니다. 
그림은 'shap' 폴더에 'summary.png'로 저장됩니다.
'''



In [ ]:
# plot_shap2 함수


def plot_shap2(svs, y_scores, cmap=plt.cm.Blues):
    # population-level interpretation
    leads = np.array(['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    n = y_scores.shape[0]
    results = [[], [], [], [], [], [], [], [], []]
    print(svs.shape)
    for i in tqdm(range(n)):
        label = np.argmax(y_scores[i])
        results[label].append(svs[label, i])
    ys = []
    for label in range(y_scores.shape[1]):
        result = np.array(results[label])
        y = []
        for i, _ in enumerate(leads):
            y.append(result[:,i].sum())
        y = np.array(y) / np.sum(y)
        ys.append(y)
        plt.plot(leads, y)
    ys.append(np.array(ys).mean(axis=0))
    ys = np.array(ys)
    fig, axs = plt.subplots()
    im = axs.imshow(ys, cmap=cmap)
    axs.figure.colorbar(im, ax=axs)
    fmt = '.2f'
    xlabels = leads
    ylabels = ['SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE'] + ['AVG']
    axs.set_xticks(np.arange(len(xlabels)))
    axs.set_yticks(np.arange(len(ylabels)))
    axs.set_xticklabels(xlabels)
    axs.set_yticklabels(ylabels)
    thresh = ys.max() / 2
    for i in range(ys.shape[0]):
        for j in range(ys.shape[1]):
            axs.text(j, i, format(ys[i, j], fmt),
                    ha='center', va='center',
                    color='white' if ys[i, j] > thresh else 'black')
    np.set_printoptions(precision=2)
    fig.tight_layout()
    plt.savefig('./shap/shap2.png')
    plt.clf()
    



    






In [ ]:
'''

제공한 코드는 plot_shap2 함수로, SHAP 값을 시각화하여 전체적인 특성 중요도를 나타내는 역할을 합니다. 이 함수는 다음과 같은 매개변수를 사용합니다:

svs: SHAP 값들을 포함하는 배열입니다.
y_scores: 예측된 클래스 점수들을 포함하는 배열입니다.
cmap: 컬러 맵(cmap)으로 사용할 Matplotlib 컬러 맵 객체입니다. 기본값은 plt.cm.Blues입니다.
함수의 동작은 다음과 같습니다:

리드(심전도 신호)의 배열(leads)을 생성합니다.
예측된 클래스 점수들(y_scores)의 개수(n)만큼 반복하면서, 각 샘플에 대한 SHAP 값을 추출합니다.
예측된 클래스 점수별로 SHAP 값을 분류하여 결과 배열(results)에 저장합니다.
각 클래스별로 리드별 SHAP 값을 합산하여 특성 중요도(y)를 계산합니다.
각 클래스별 특성 중요도(y)를 시각화하고, 평균 특성 중요도도 추가하여 시각화 데이터(ys)를 생성합니다.
특성 중요도(ys)를 히트맵으로 시각화합니다.
히트맵에 컬러 바(colorbar)를 추가합니다.
히트맵의 셀에 숫자를 표시하고, 숫자의 색상을 임계값(thresh)을 기준으로 지정합니다.
축 눈금과 레이블을 설정합니다.
그림을 './shap/shap2.png' 파일로 저장합니다.
그림을 초기화합니다.
이 함수를 사용하면 SHAP 값을 시각화하여 전체적인 특성 중요도를 파악할 수 있습니다. 히트맵과 숫자로 표현된 특성 중요도는 'shap' 폴더에 'shap2.png'로 저장됩니다.


'''

In [ ]:
# main 블록

if __name__ == '__main__':
    args = parse_args()
    data_dir = os.path.normpath(args.data_dir)
    database = os.path.basename(data_dir)
    args.model_path = f'models/resnet34_{database}_{args.leads}_{args.seed}.pth'
    label_csv = os.path.join(data_dir, 'labels.csv')
    reference_csv = os.path.join(data_dir, 'reference.csv')
    lleads = np.array(['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    classes = np.array(['SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE'])
    if args.use_gpu and torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = 'cpu'
    if args.leads == 'all':
        leads = 'all'
        nleads = 12
    else:
        leads = args.leads.split(',')
        nleads = len(leads)
    
    model = resnet34(input_channels=nleads).to(device)
    model.load_state_dict(torch.load(args.model_path, map_location=device))
    model.eval()

    background = 100
    result_path = f'results/A{background * 2}.npy'

    df_labels = pd.read_csv(label_csv)
    df_reference = pd.read_csv(os.path.join(args.data_dir, 'reference.csv'))
    df = pd.merge(df_labels, df_reference[['patient_id', 'age', 'sex', 'signal_len']], on='patient_id', how='left')

    # df = df[df['signal_len'] >= 15000]

    patient_ids = df['patient_id'].to_numpy()
    to_explain = patient_ids[:background * 2]

    background_patient_ids = df.head(background)['patient_id'].to_numpy()
    background_inputs = [os.path.join(data_dir, patient_id) for patient_id in background_patient_ids]
    background_inputs = torch.stack([torch.from_numpy(prepare_input(input)).float() for input in background_inputs]).to(device)
    
    e = shap.GradientExplainer(model, background_inputs)

    if not os.path.exists(result_path):
        svs = []
        y_scores = []
        for patient_id in tqdm(to_explain):
            input = os.path.join(data_dir, patient_id)
            inputs = torch.stack([torch.from_numpy(prepare_input(input)).float()]).to(device)
            y_scores.append(torch.sigmoid(model(inputs)).detach().cpu().numpy())
            sv = np.array(e.shap_values(inputs)) # (n_classes, n_samples, n_leads, n_points)
            svs.append(sv)
        svs = np.concatenate(svs, axis=1)
        y_scores = np.concatenate(y_scores, axis=0)
        np.save(result_path, (svs, y_scores))
    svs, y_scores = np.load(result_path, allow_pickle=True)

    # summary_plot(svs, y_scores)
    plot_shap2(svs, y_scores)

    preds = []
    top_leads_list = []
    for i, patient_id in enumerate(to_explain):
        ecg_data = prepare_input(os.path.join(data_dir, patient_id))
        label_idx = np.argmax(y_scores[i])
        sv_data = svs[label_idx, i]
        
        sv_data_mean = np.mean(sv_data, axis=1)
        top_leads = np.where(sv_data_mean > 1e-4)[0] # select top leads
        preds.append(classes[label_idx])
        print(patient_id, classes[label_idx], lleads[top_leads])

        plot_shap(ecg_data, sv_data, top_leads, patient_id, classes[label_idx])



In [ ]:

'''

주어진 코드는 주어진 데이터를 사용하여 모델의 SHAP 값을 시각화하는 기능을 가지고 있습니다. 코드의 내용은 다음과 같습니다:

명령줄 인수를 파싱하여 사용할 데이터 디렉토리 및 다른 매개변수를 설정합니다.
데이터 디렉토리와 관련된 정보를 설정합니다.
필요한 라이브러리와 모델을 임포트합니다.
모델과 SHAP 값을 계산하기 위한 데이터를 준비합니다.
SHAP 값을 계산하고 시각화합니다.
코드를 실행할 때, 다음과 같은 작업이 수행됩니다:

명령줄 인수를 사용하여 데이터 디렉토리와 모델 경로를 설정합니다.
데이터 및 모델을 로드하고 모델을 평가 모드로 설정합니다.
백그라운드 데이터를 사용하여 SHAP 값을 계산합니다.
계산된 SHAP 값을 저장하고 로드합니다.
SHAP 값을 사용하여 전체적인 특성 중요도를 시각화합니다.
각 환자에 대해 예측을 수행하고 SHAP 값을 사용하여 특성을 시각화합니다.
이 코드를 실행하면 SHAP 값을 계산하고 시각화하여 모델의 예측을 해석할 수 있습니다. 결과는 './shap' 폴더에 저장됩니다.



'''
